In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.add(1, 2).numpy()
3
hello = tf.constant('Hello, TensorFlow!')
hello.numpy()
'Hello, TensorFlow!'

# CNN Examples

In [ ]:
!cd /workspace && ls -lag

In [ ]:
!cd /workspace/nvidia-examples/ && ls -lag

In [ ]:
!cd /workspace/nvidia-examples/cnn/ && ls -lag

In [1]:
cd /workspace/nvidia-examples/cnn/

/opt/tensorflow/nvidia-examples/cnn


# Alexnet

In [2]:
!cat ./alexnet.py

#!/usr/bin/env python
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import print_function
from builtins import range
import nvutils
import tensorflow as tf

nvutils.init()

default_args = {
    'image_width' : 227,
    'image_height' : 227,
    'image_format' : 'channels_first',
    'distort_color' : False,
    'batch_size' : 256,
    'da

In [3]:
!./alexnet.py 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --display_every 10
  --num_iter 91
  --precision fp16
  --batch_size 256
  --predict False
  --iter_unit epoch
Training
  Step Epoch Img/sec   Loss  LR
     1   1.0    76.7  6.922  9.996 1.00000
    10  10.0   844.0  5.879  8.943 0.81198
    20  20.0  6629.0  5.683  8.728 0.62601
    30  30.0  6630.8  5.605  8.619 0.46420
    40  40.0  6665.7  5.585  8.559 0.32653
    50  50.0  6638.2  5.553  8.489 0.21302
    60  60.0  6643.6  5.529  8.434 0.12366
    70  70.0  6645.9  5.492  8.378 0.05845
    80  80.0  6628.7  5.475  8.351 0.01739
    90  90.0  6641.2  5.427  8.302 0.00048


In [4]:
!./alexnet.py --data_dir /datasets/imagenet_TFrecords 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --num_iter 91
  --batch_size 256
  --predict False
  --precision fp16
  --data_dir /datasets/imagenet_TFrecords
  --display_every 10
  --iter_unit epoch
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    80.1  6.919  9.993 1.00000
    10   0.0   778.1  6.911  9.951 0.99996
    20   0.0  4273.9  6.916  9.836 0.99992
    30   0.0  4476.0  6.909  9.620 0.99987
    40   0.0  3934.0  6.902  9.379 0.99983
    50   0.0  4118.3  6.909  9.157 0.99978
    60   0.0  4210.7  6.909  8.941 0.99974
    70   0.0  4482.8  6.905  8.740 0.99970
    80   0.0  4387.3  6.913  8.568 0.99965
    90   0.0  4489.4  6.905  8.397 0.99961
   100   0.0  4508.3  6.911  8.255 0.99957
   110   0.0  4204.9  6.910  8.122 0.99952
   120   0.0  4523.4  6.911  8.003 0.99948
   130   0.0  4543.2  6.909  7.893 0.99943
   140   0.0  4480.6  6.911  7.798 0.99939
   150   0.0  4088.5  6.908  7.707 0.99935
   160   0.0  4004.3  6.

  1840   0.4  4542.2  6.519  6.632 0.99194
  1850   0.4  4636.6  6.595  6.708 0.99190
  1860   0.4  4724.7  6.555  6.668 0.99185
  1870   0.4  4350.0  6.627  6.741 0.99181
  1880   0.4  4096.4  6.591  6.705 0.99176
  1890   0.4  4373.1  6.649  6.763 0.99172
  1900   0.4  4037.1  6.609  6.722 0.99168
  1910   0.4  4033.8  6.511  6.624 0.99163
  1920   0.4  4035.8  6.579  6.693 0.99159
  1930   0.4  4232.3  6.453  6.567 0.99155
  1940   0.4  4641.9  6.529  6.643 0.99150
  1950   0.4  4549.8  6.525  6.638 0.99146
  1960   0.4  4121.4  6.457  6.571 0.99141
  1970   0.4  4134.7  6.540  6.653 0.99137
  1980   0.4  4533.2  6.519  6.633 0.99133
  1990   0.4  4599.4  6.548  6.661 0.99128
  2000   0.4  4641.4  6.312  6.426 0.99124
  2010   0.4  4395.7  6.521  6.635 0.99120
  2020   0.4  4376.6  6.473  6.587 0.99115
  2030   0.4  4538.2  6.547  6.661 0.99111
  2040   0.4  4504.6  6.479  6.592 0.99106
  2050   0.4  4123.7  6.435  6.549 0.99102
  2060   0.4  4041.6  6.513  6.627 0.99098
  2070   0.

# InceptionV3

In [5]:
!./inception_v3.py --data_dir /datasets/imagenet_TFrecords 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --predict False
  --data_dir /datasets/imagenet_TFrecords
  --precision fp16
  --display_every 10
  --num_iter 90
  --iter_unit epoch
  --batch_size 128
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0     9.5  7.037  8.149 1.00000
    10   0.0    69.2  7.241  8.355 0.99998
    20   0.0   460.8  7.128  8.244 0.99996
    30   0.0   461.4  7.027  8.142 0.99994
    40   0.0   461.4  6.953  8.062 0.99991
    50   0.0   465.1  6.984  8.082 0.99989
    60   0.0   465.0  6.961  8.044 0.99987
    70   0.0   465.4  6.946  8.013 0.99985
    80   0.0   464.5  6.922  7.971 0.99982
    90   0.0   462.1  6.885  7.918 0.99980
   100   0.0   463.7  6.907  7.924 0.99978
   110   0.0   458.5  6.910  7.909 0.99976
   120   0.0   463.2  6.857  7.839 0.99974
   130   0.0   462.3  6.804  7.768 0.99971
^C


# Resnet

In [6]:
!./resnet.py --layers=50 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --precision fp16
  --display_every 10
  --batch_size 256
  --layers 50
  --num_iter 90
  --iter_unit epoch
  --predict False
Training
  Step Epoch Img/sec   Loss  LR
     1   1.0    24.5  7.660  8.631 2.00000
    10  10.0   269.3  4.229  5.201 1.62000
    20  20.0   867.2  0.059  1.036 1.24469
    30  30.0   866.0  0.076  1.057 0.91877
    40  40.0   863.5  0.075  1.058 0.64222
    50  50.0   863.8  0.079  1.064 0.41506
    60  60.0   862.5  0.186  1.172 0.23728
    70  70.0   861.2  0.048  1.036 0.10889
    80  80.0   864.8  0.001  0.990 0.02988
    90  90.0   650.6  0.001  0.989 0.00025


In [7]:
!./resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --num_iter 90
  --layers 50
  --display_every 10
  --iter_unit epoch
  --data_dir /datasets/imagenet_TFrecords
  --predict False
  --batch_size 256
  --precision fp16
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    25.0  7.601  8.572 2.00000
    10   0.0   226.7  7.568  8.540 1.99992
    20   0.0   765.2  7.120  8.095 1.99983
    30   0.0   861.9  7.060  8.039 1.99974
    40   0.0   862.1  7.085  8.062 1.99965
    50   0.0   859.9  7.073  8.047 1.99956
    60   0.0   856.4  6.945  7.913 1.99948
    70   0.0   863.3  6.982  7.941 1.99939
    80   0.0   861.3  6.916  7.862 1.99930
    90   0.0   866.3  6.854  7.784 1.99921
   100   0.0   865.5  6.889  7.802 1.99912
   110   0.0   860.1  6.924  7.824 1.99903
   120   0.0   863.2  6.818  7.702 1.99894
   130   0.0   862.1  6.935  7.801 1.99885
^C


In [11]:
!./resnet.py --layers=50 --batch_size=256 --data_dir=/datasets/imagenet_TFrecords --precision=fp32 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --predict False
  --layers 50
  --batch_size 256
  --precision fp32
  --display_every 10
  --num_iter 90
  --iter_unit epoch
  --data_dir /datasets/imagenet_TFrecords
Training
  Step Epoch Img/sec   Loss  LR


In [9]:
!./resnet.py --layers=50 --batch_size=128 --data_dir=/datasets/imagenet_TFrecords --precision=fp32 2>/tmp/errors

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --precision fp32
  --display_every 10
  --predict False
  --layers 50
  --num_iter 90
  --batch_size 128
  --iter_unit epoch
  --data_dir /datasets/imagenet_TFrecords
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    16.3  7.571  8.543 2.00000
    10   0.0   121.3  8.120  9.092 1.99996
    20   0.0   363.9  7.674  8.650 1.99992
    30   0.0   392.8  7.400  8.380 1.99987
    40   0.0   392.8  7.103  8.085 1.99983
    50   0.0   391.5  7.004  7.984 1.99978
    60   0.0   392.0  6.990  7.964 1.99974
    70   0.0   391.1  6.948  7.909 1.99969
    80   0.0   392.9  6.971  7.917 1.99965
    90   0.0   389.5  6.933  7.863 1.99960
   100   0.0   392.7  6.961  7.872 1.99956
   110   0.0   392.2  7.180  8.071 1.99952
   120   0.0   394.7  6.979  7.855 1.99947
   130   0.0   392.9  6.905  7.762 1.99943
   140   0.0   393.0  6.976  7.810 1.99938
   150   0.0   393.0  6.888  7.699 1.99934
   160   0

# DALI with Resnet

In [10]:
!./resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords --precision=fp16 --data_idx_dir /datasets/imagenet_idx --use_dali GPU 2>/tmp/error

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --num_iter 90
  --display_every 10
  --data_idx_dir /datasets/imagenet_idx
  --iter_unit epoch
  --use_dali GPU
  --predict False
  --layers 50
  --batch_size 256
  --data_dir /datasets/imagenet_TFrecords
  --precision fp16
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    22.4  7.700  8.671 2.00000
    10   0.0   248.6  7.694  8.666 1.99992
    20   0.0   810.5  7.415  8.390 1.99983
    30   0.0   845.9  7.405  8.382 1.99974
    40   0.0   843.7  7.085  8.065 1.99965
    50   0.0   833.9  6.991  7.966 1.99956
    60   0.0   842.5  6.949  7.915 1.99948
    70   0.0   840.9  7.771  8.724 1.99939
    80   0.0   842.5  6.929  7.873 1.99930
    90   0.0   843.1  7.002  7.937 1.99921
   100   0.0   839.9  6.883  7.808 1.99912
   110   0.0   837.3  6.926  7.839 1.99903
   120   0.0   846.5  6.859  7.754 1.99894
   130   0.0   833.7  6.779  7.652 1.99885
   140   0.0   843.2  6.834  7.684 1.99

# DALI with Alexnet

In [16]:
!./alexnet.py --batch_size 256 --data_dir=/datasets/imagenet_TFrecords --precision=fp16 --data_idx_dir /datasets/imagenet_idx --use_dali GPU 2>/tmp/error

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --num_iter 91
  --data_dir /datasets/imagenet_TFrecords
  --display_every 10
  --predict False
  --use_dali GPU
  --batch_size 256
  --iter_unit epoch
  --data_idx_dir /datasets/imagenet_idx
  --precision fp16
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0    61.6  6.912  9.986 1.00000
    10   0.0   745.1  6.927  9.968 0.99996
    20   0.0  5591.9  6.910  9.802 0.99992
    30   0.0  5590.1  6.912  9.583 0.99987
    40   0.0  5558.6  6.915  9.348 0.99983
    50   0.0  5535.5  6.899  9.102 0.99978
    60   0.0  5547.8  6.907  8.896 0.99974
    70   0.0  5557.9  6.916  8.711 0.99970
    80   0.0  5582.6  6.900  8.519 0.99965
    90   0.0  5547.7  6.907  8.368 0.99961
   100   0.0  5598.3  6.908  8.225 0.99957
   110   0.0  5577.5  6.907  8.094 0.99952
   120   0.0  5626.7  6.907  7.978 0.99948
   130   0.0  5612.0  6.908  7.877 0.99943
   140   0.0  5573.5  6.910  7.786 0.99939
   150   0

# Multi GPU with Resnet

In [17]:
!mpiexec --allow-run-as-root --bind-to socket -np 8 /workspace/nvidia-examples/cnn/resnet.py --layers=50 --data_dir=/datasets/imagenet_TFrecords --precision=fp16 --data_idx_dir /datasets/imagenet_idx --use_dali GPU 2>/tmp/error

PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
PY 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
TF 1.12.0
Script arguments:
  --display_every 10
  --precision fp16
  --num_iter 90
  --use_dali GPU
  --iter_unit epoch
  --batch_size 256
  --predict False
  --data_idx_dir /datasets/imagenet_idx
  --data_dir /datasets/imagenet_TFrecords
  --layers 50
Training
Training
Training
Training
Training
Training
Training
Training
  Step Epoch Img/sec   Loss  LR
     1   0.0   145.2  7.633  8.605 2.00000
    10   0.0  152